# Numerical widgets

## Defining a Slider Widget

In [1]:
#pragma cling add_library_path("/opt/conda/lib/")
#pragma cling add_include_path("/opt/conda/include/")
#pragma cling add_include_path("/opt/conda/include/xwidgets")
#pragma cling load("/opt/conda/lib/libxwidgets.so.1.0.1")

In [2]:

#include <xslider.hpp>


In [3]:
/***************************************************************************
 * Copyright (c) 2022, QuantStack and XWidgets contributors                 *
 *                                                                          *
 * Distributed under the terms of the BSD 3-Clause License.                 *
 *                                                                          *
 * The full license is in the file LICENSE, distributed with this software. *
 ****************************************************************************/

#include "xwidgets/xcommon.hpp"

#include <algorithm>
#include <cstdlib>
#include <string>
#include <utility>
#include <vector>

#include <xtl/xoptional.hpp>

#include "xeus/xinterpreter.hpp"
#include "xtarget.hpp"

namespace xw
{
    xcommon::xcommon()
        : m_moved_from(false)
        , m_hold(nullptr)
        , m_comm(get_widget_target(), xeus::new_xguid())
    {
    }

    xcommon::~xcommon()
    {
    }

    xcommon::xcommon(xeus::xcomm&& comm)
        : m_moved_from(false)
        , m_hold(nullptr)
        , m_comm(std::move(comm))
    {
    }

    xcommon::xcommon(const xcommon& other)
        : m_moved_from(false)
        , m_hold(nullptr)
        , m_comm(other.m_comm)
        , m_buffer_paths(other.m_buffer_paths)
    {
    }

    xcommon::xcommon(xcommon&& other)
        : m_moved_from(false)
        , m_hold(nullptr)
        , m_comm(std::move(other.m_comm))
        , m_buffer_paths(std::move(other.m_buffer_paths))
    {
        other.m_moved_from = true;
    }

    xcommon& xcommon::operator=(const xcommon& other)
    {
        m_moved_from = false;
        m_hold = nullptr;
        m_comm = other.m_comm;
        m_buffer_paths = other.m_buffer_paths;
        return *this;
    }

    xcommon& xcommon::operator=(xcommon&& other)
    {
        other.m_moved_from = true;
        m_moved_from = false;
        m_hold = nullptr;
        m_comm = std::move(other.m_comm);
        m_buffer_paths = std::move(other.m_buffer_paths);
        return *this;
    }

    auto xcommon::id() const noexcept -> xeus::xguid
    {
        return m_comm.id();
    }

    void xcommon::display() const
    {
        nl::json mime_bundle;

        // application/vnd.jupyter.widget-view+json
        nl::json widgets_json;
        widgets_json["version_major"] = XWIDGETS_PROTOCOL_VERSION_MAJOR;
        widgets_json["version_minor"] = XWIDGETS_PROTOCOL_VERSION_MINOR;
        widgets_json["model_id"] = this->id();
        mime_bundle["application/vnd.jupyter.widget-view+json"] = std::move(widgets_json);

        // text/plain
        mime_bundle["text/plain"] = "A Jupyter widget";

        ::xeus::get_interpreter().display_data(std::move(mime_bundle), nl::json::object(), nl::json::object());
    }

    void xcommon::send(nl::json&& content, xeus::buffer_sequence&& buffers) const
    {
        // metadata
        nl::json metadata;
        metadata["version"] = XWIDGETS_PROTOCOL_VERSION;

        // data
        nl::json data;
        data["method"] = "custom";
        data["content"] = std::move(content);

        // send
        m_comm.send(std::move(metadata), std::move(data), std::move(buffers));
    }

    void xcommon::handle_custom_message(const nl::json& /*content*/)
    {
    }

    xeus::xcomm& xcommon::comm()
    {
        return m_comm;
    }

    const xeus::xcomm& xcommon::comm() const
    {
        return m_comm;
    }

    const xeus::xmessage*& xcommon::hold()
    {
        return m_hold;
    }

    const xeus::xmessage* const& xcommon::hold() const
    {
        return m_hold;
    }

    bool xcommon::moved_from() const noexcept
    {
        return m_moved_from;
    }

    std::vector<xjson_path_type>& xcommon::buffer_paths()
    {
        return m_buffer_paths;
    }

    const std::vector<xjson_path_type>& xcommon::buffer_paths() const
    {
        return m_buffer_paths;
    }

    void xcommon::send_patch(nl::json&& patch, xeus::buffer_sequence&& buffers, const char* method) const
    {
        // extract buffer paths
        std::vector<nl::json> paths{};
        reorder_buffer_paths(buffer_paths(), patch, paths);

        // metadata
        nl::json metadata;
        metadata["version"] = XWIDGETS_PROTOCOL_VERSION;

        // data
        nl::json data;
        data["method"] = method;
        data["state"] = std::move(patch);
        data["buffer_paths"] = std::move(paths);

        // send
        m_comm.send(std::move(metadata), std::move(data), std::move(buffers));
    }

    void xcommon::open(nl::json&& patch, xeus::buffer_sequence&& buffers)
    {
        // extract buffer paths
        std::vector<nl::json> paths{};
        reorder_buffer_paths(buffer_paths(), patch, paths);

        // metadata
        nl::json metadata;
        metadata["version"] = XWIDGETS_PROTOCOL_VERSION;

        // data
        nl::json data;

        data["state"] = std::move(patch);
        data["buffer_paths"] = std::move(paths);

        // open
        m_comm.open(std::move(metadata), std::move(data), std::move(buffers));
    }

    void xcommon::close()
    {
        // close
        m_comm.close(nl::json::object(), nl::json::object(), xeus::buffer_sequence());
    }

    namespace
    {
        std::string tolower(std::string s)
        {
            auto safe_tolower = [](unsigned char c)
            {
                return std::tolower(c);
            };
            std::transform(s.begin(), s.end(), s.begin(), safe_tolower);
            return s;
        }

        std::string ltrim(std::string s)
        {
            auto const safe_isnotspace = [](unsigned char ch)
            {
                return !std::isspace(ch);
            };
            s.erase(s.begin(), std::find_if(s.begin(), s.end(), safe_isnotspace));
            return s;
        }

        std::string rtrim(std::string s)
        {
            auto const safe_isnotspace = [](unsigned char ch)
            {
                return !std::isspace(ch);
            };
            s.erase(std::find_if(s.rbegin(), s.rend(), safe_isnotspace).base(), s.end());
            return s;
        }

        std::string trim(std::string s)
        {
            rtrim(s);
            ltrim(s);
            return s;
        }

        bool is_zero_number(std::string const& s)
        {
            try
            {
                auto val = std::stod(s);
                return val == decltype(val)(0);
            }
            catch (std::invalid_argument const&)
            {
                return false;
            }
            catch (std::out_of_range const&)
            {
                return false;
            }
        }

        /**
         * Check if a string is true.
         *
         * A string is true, if it does not contain a negative value, such as "false" or 0.
         * This is the convention adopted in Jupyter
         * https://github.com/jupyter/jupyter_core/blob/98ab1ef453956333a85bb6eee494ad0a9bab2c02/jupyter_core/paths.py#L47
         */
        bool is_true_string(const char* str)
        {
            const std::string s = tolower(trim(str));
            if (is_zero_number(str))
            {
                return false;
            }
            static constexpr auto falses = {"no", "n", "false", "off"};
            return std::find(falses.begin(), falses.end(), s) == falses.end();
        }

        xtl::xoptional<bool> get_tristate_env(const char* name)
        {
            const char* const val = std::getenv(name);
            if (val == nullptr)
            {
                return {};
            }
            return is_true_string(val);
        }
    }

    xtl::xoptional<bool> xcommon::global_echo_update()
    {
        static const auto out = get_tristate_env("JUPYTER_WIDGETS_ECHO");
        return out;
    }

    bool
    xcommon::same_patch(const std::string& name, const nl::json& j1, const xeus::buffer_sequence&, const nl::json& j2, const xeus::buffer_sequence&)
        const
    {
        const auto& paths = buffer_paths();
        // For a widget with no binary buffer, compare the patches
        if (paths.empty())
        {
            return j1 == j2;
        }
        else
        {
            // For a property with no binary buffer, compare the patches
            if (std::find_if(
                    paths.cbegin(),
                    paths.cend(),
                    [name](const auto& v)
                    {
                        return !v.empty() && v[0] == name;
                    }
                )
                == paths.cend())
            {
                return j1 == j2;
            }
            else
            {
                // TODO: handle the comparison of binary buffers.
                return true;
            }
        }
    }

    void to_json(nl::json& j, const xcommon& o)
    {
        j = "IPY_MODEL_" + std::string(o.id());
    }

    nl::json mime_bundle_repr(const xcommon& val)
    {
        nl::json mime_bundle;

        // application/vnd.jupyter.widget-view+json
        nl::json widgets_json;
        widgets_json["version_major"] = XWIDGETS_PROTOCOL_VERSION_MAJOR;
        widgets_json["version_minor"] = XWIDGETS_PROTOCOL_VERSION_MINOR;
        widgets_json["model_id"] = val.id();
        mime_bundle["application/vnd.jupyter.widget-view+json"] = std::move(widgets_json);

        // text/plain
        mime_bundle["text/plain"] = "A Jupyter widget with unique id: " + std::string(val.id());
        return mime_bundle;
    }
}

In [4]:
/***************************************************************************
 * Copyright (c) 2017, Sylvain Corlay and Johan Mabille                     *
 *                                                                          *
 * Distributed under the terms of the BSD 3-Clause License.                 *
 *                                                                          *
 * The full license is in the file LICENSE, distributed with this software. *
 ****************************************************************************/

#include "xwidgets/xbinary.hpp"

#include <sstream>
#include <string>
#include <utility>

namespace nl = nlohmann;

namespace xw
{
    const std::string& xbuffer_reference_prefix()
    {
        static const std::string prefix = "@buffer_reference@";
        return prefix;
    }

    bool is_buffer_reference(const std::string& arg)
    {
        const std::string& prefix = xbuffer_reference_prefix();
        return arg.size() > prefix.size() && std::equal(prefix.cbegin(), prefix.cend(), arg.cbegin());
    }

    std::size_t buffer_index(const std::string& v)
    {
        std::stringstream stream(v);
        auto const prefix_size = xbuffer_reference_prefix().size();
        stream.ignore(static_cast<std::streamsize>(prefix_size));
        std::size_t index = 0;
        stream >> index;
        return index;
    }

    namespace detail
    {
        const nl::json* get_buffers(const nl::json& patch, const xjson_path_type& path)
        {
            const nl::json* current = &patch;
            for (const auto& item : path)
            {
                if (current->is_array())
                {
                    current = &(*current)[std::stoul(item)];
                }
                else
                {
                    auto el = current->find(item);
                    if (el != current->end())
                    {
                        current = &(*el);
                    }
                    else
                    {
                        return nullptr;
                    }
                }
            }
            return current;
        }

        nl::json* get_json(nl::json& patch, const xjson_path_type& path)
        {
            nl::json* current = &patch;
            for (const auto& item : path)
            {
                if (current->is_array())
                {
                    current = &(*current)[std::stoul(item)];
                }
                else
                {
                    current = &(*current)[item];
                }
            }
            return current;
        }

        template <class T>
        void set_json(nl::json& patch, const xjson_path_type& path, const T& value)
        {
            nl::json* json = get_json(patch, path);
            if (json != nullptr)
            {
                (*json) = value;
            }
        }

        void insert_buffer_path(nl::json& patch, const nl::json& path, std::size_t buffer_index)
        {
            xjson_path_type p = path;
            detail::set_json(patch, p, xbuffer_reference_prefix() + std::to_string(buffer_index));
        }
    }

    void reorder_buffer_paths(
        const std::vector<xjson_path_type>& buffer_paths,
        const nl::json& patch,
        std::vector<nl::json>& out
    )
    {
        auto ensure_out_size = [&out](std::size_t size)
        {
            if (out.size() < size)
            {
                out.resize(size, nullptr);
            }
        };

        ensure_out_size(buffer_paths.size());
        for (const auto& path : buffer_paths)
        {
            const nl::json* item = detail::get_buffers(patch, path);
            if (item != nullptr && item->is_string())
            {
                const auto& leaf = item->get<std::string>();
                if (is_buffer_reference(leaf))
                {
                    auto const idx = buffer_index(leaf);
                    // Idx may be greater than to_check.size() when the buffers are used with
                    // multiple states
                    ensure_out_size(idx + 1);
                    out[idx] = path;
                }
            }
        }
    }

    void insert_buffer_paths(nl::json& patch, const nl::json& buffer_paths)
    {
        for (std::size_t i = 0; i != buffer_paths.size(); ++i)
        {
            detail::insert_buffer_path(patch, buffer_paths[i], i);
        }
    }
}

In [5]:
/***************************************************************************
 * Copyright (c) 2017, Sylvain Corlay and Johan Mabille                     *
 *                                                                          *
 * Distributed under the terms of the BSD 3-Clause License.                 *
 *                                                                          *
 * The full license is in the file LICENSE, distributed with this software. *
 ****************************************************************************/

#include "xwidgets/xfactory.hpp"

#include <utility>

namespace nl = nlohmann;

namespace xw
{
    void xfactory::register_maker(
        const std::string& model_module,
        const std::string& model_name,
        const std::string& view_module,
        const std::string& view_name,
        maker_type maker
    )
    {
        m_makers[model_module + model_name + view_module + view_name] = std::move(maker);
    }

    void xfactory::make(xeus::xcomm&& comm, const nl::json& state, const xeus::buffer_sequence& buffers) const
    {
        std::string model_module = state["_model_module"];
        std::string model_name = state["_model_name"];
        std::string view_module = state["_view_module"];
        std::string view_name = state["_view_name"];
        m_makers.at(model_module + model_name + view_module + view_name)(std::move(comm), state, buffers);
    }

    xfactory& get_xfactory()
    {
        static xfactory factory;
        return factory;
    }
}

In [6]:
#include "xwidgets/xholder.hpp"

#include <string>
#include <utility>

namespace xw
{
    xholder::xholder()
        : p_holder(nullptr)
    {
    }

    xholder::xholder(detail::xholder_impl* holder)
        : p_holder(holder)
    {
    }

    xholder::~xholder()
    {
        delete p_holder;
    }

    xholder::xholder(const xholder& rhs)
        : p_holder(rhs.p_holder ? rhs.p_holder->clone() : nullptr)
    {
    }

    xholder::xholder(xholder&& rhs)
        : p_holder(rhs.p_holder)
    {
        rhs.p_holder = nullptr;
    }

    xholder& xholder::operator=(const xholder& rhs)
    {
        using std::swap;
        xholder tmp(rhs);
        swap(*this, tmp);
        return *this;
    }

    xholder& xholder::operator=(xholder&& rhs)
    {
        using std::swap;
        xholder tmp(std::move(rhs));
        swap(*this, tmp);
        return *this;
    }

    void xholder::swap(xholder& rhs)
    {
        std::swap(p_holder, rhs.p_holder);
    }

    void xholder::display() const
    {
        check_holder();
        p_holder->display();
    }

    xeus::xguid xholder::id() const
    {
        check_holder();
        return p_holder->id();
    }

    void xholder::serialize_state(nl::json& state, xeus::buffer_sequence& buffers) const
    {
        check_holder();
        return p_holder->serialize_state(state, buffers);
    }

    const std::vector<xjson_path_type>& xholder::buffer_paths() const
    {
        check_holder();
        return p_holder->buffer_paths();
    }

    xtl::any xholder::value() &
    {
        check_holder();
        return p_holder->value();
    }

    const xtl::any xholder::value() const&
    {
        check_holder();
        return p_holder->value();
    }

    void xholder::check_holder() const
    {
        if (p_holder == nullptr)
        {
            throw std::runtime_error("The holder does not contain a widget");
        }
    }

    void swap(xholder& lhs, xholder& rhs)
    {
        lhs.swap(rhs);
    }

    /****************************************
     * to_json and from_json implementation *
     ****************************************/

    void to_json(nl::json& j, const xholder& o)
    {
        j = "IPY_MODEL_" + std::string(o.id());
    }

    void from_json(const nl::json& j, xholder& o)
    {
        std::string prefixed_guid = j;
        xeus::xguid guid = prefixed_guid.substr(10).c_str();
        o = make_id_holder(guid);
    }

    nl::json mime_bundle_repr(const xholder& val)
    {
        nl::json mime_bundle;

        // application/vnd.jupyter.widget-view+json
        nl::json widgets_json;
        widgets_json["version_major"] = XWIDGETS_PROTOCOL_VERSION_MAJOR;
        widgets_json["version_minor"] = XWIDGETS_PROTOCOL_VERSION_MINOR;
        widgets_json["model_id"] = val.id();
        mime_bundle["application/vnd.jupyter.widget-view+json"] = std::move(widgets_json);

        // text/plain
        mime_bundle["text/plain"] = "A Jupyter widget";
        return mime_bundle;
    }
}

In [7]:
/***************************************************************************
 * Copyright (c) 2017, Sylvain Corlay and Johan Mabille                     *
 *                                                                          *
 * Distributed under the terms of the BSD 3-Clause License.                 *
 *                                                                          *
 * The full license is in the file LICENSE, distributed with this software. *
 ****************************************************************************/

#include "xtarget.hpp"

#include <algorithm>
#include <array>
#include <stdexcept>
#include <string>
#include <unordered_map>
#include <utility>

#include <nlohmann/json.hpp>
#include <xeus/xcomm.hpp>
#include <xeus/xinterpreter.hpp>

#include "xwidgets/xbinary.hpp"
#include "xwidgets/xcommon.hpp"
#include "xwidgets/xfactory.hpp"
#include "xwidgets/xregistry.hpp"
#include "xwidgets/xwidgets_config.hpp"

namespace xw
{
    namespace
    {
        const char* get_widget_target_name()
        {
            return "jupyter.widget";
        }

        /**
         * Check frontend widget version and instanciate widget.
         *
         * This callback function is called by Xeus when a comm channel is open by the frontend
         * to create a companion widget in the kernel.
         */
        void xobject_comm_opened(xeus::xcomm&& comm, const xeus::xmessage& msg)
        {
            const nl::json& content = msg.content();
            const nl::json& metadata = msg.metadata();

            std::string version;
            try
            {
                version = metadata.at("version").get<std::string>();
            }
            catch (std::out_of_range const&)
            {
                version = "";
            }

            if (version.substr(0, version.find(".")) != XWIDGETS_STRINGIFY(XWIDGETS_PROTOCOL_VERSION_MAJOR))
            {
                throw std::runtime_error("Incompatible widget protocol versions");
            }

            const nl::json& data = content["data"];
            const nl::json& state = data["state"];
            const xeus::buffer_sequence& buffers = msg.buffers();

            xfactory& factory = get_xfactory();
            factory.make(std::move(comm), state, buffers);
        }

        void register_widget_target()
        {
            xeus::get_interpreter().comm_manager().register_comm_target(
                /** The target name */
                get_widget_target_name(),
                /** Callback for comm opened by the frontend on this target, one per widget */
                xobject_comm_opened
            );
        }
    }

    xeus::xtarget* get_widget_target()
    {
        // Making a dummy static variable to only call the registration once.
        static const auto initialized = []()
        {
            register_widget_target();
            return true;
        }();
        return ::xeus::get_interpreter().comm_manager().target(get_widget_target_name());
    }

    namespace
    {
        template <typename JsonPath>
        std::vector<xjson_path_type>
        prepend_to_json_paths(std::vector<xjson_path_type> paths, JsonPath const& prefix)
        {
            std::for_each(
                paths.begin(),
                paths.end(),
                [&](xjson_path_type& p)
                {
                    p.insert(p.begin(), prefix.begin(), prefix.end());
                }
            );
            return paths;
        }

        void
        serialize_all_states(nl::json& states, xeus::buffer_sequence& buffers, std::vector<nl::json>& buffer_paths)
        {
            for (auto const& id_and_widget : get_transport_registry())
            {
                auto const& holder = id_and_widget.second;
                // This is not what the protocol states (?) but what IPyWidgets does
                // https://github.com/jupyter-widgets/ipywidgets/issues/3685
                nl::json stateish = nl::json::object();
                holder.serialize_state(stateish["state"], buffers);
                stateish["model_name"] = stateish["state"]["_model_name"];
                stateish["model_module"] = stateish["state"]["_model_module"];
                stateish["model_module_version"] = stateish["state"]["_model_module_version"];
                states[holder.id()] = std::move(stateish);
                // Add buffer paths, but add the xguid/state prefix of multi-state schema
                reorder_buffer_paths(
                    prepend_to_json_paths(holder.buffer_paths(), std::array<std::string, 2>{holder.id(), "state"}),
                    states,
                    buffer_paths
                );
            }
        }

        /**
         * Register the ``on_message`` callback on the comm to get all widgets states.
         *
         * This callback function is called by Xeus when a comm channel is open by the frontend
         * on the ``jupyter.widget.control`` target.
         * This happens when the frontend needs to get the state of all widgets and no immediate
         * action is required.
         * Following the opening of the comm, the frontend sends a message with a
         * ``request_states`` method, to which the kernel replies with the state of all widgets.
         *
         * After the frontend recieves the ``update_states`` response it closes the comm.
         * Additional (and simulataneous) comms can be opened for fetching states.
         */
        void control_comm_opened(xeus::xcomm&& comm, const xeus::xmessage&)
        {
            // This is a very simple registry for comm since their lifetime is managed by the
            // frontend
            static std::unordered_map<xeus::xguid, xeus::xcomm> comm_registry{};

            auto iter_inserted = comm_registry.emplace(std::make_pair(comm.id(), std::move(comm)));
            // Should really be inserted, but in case it is not, we let the comm gets destroyed and closed
            assert(iter_inserted.second);
            if (!iter_inserted.second)
            {
                return;
            }

            auto& registered_comm = iter_inserted.first->second;

            registered_comm.on_message(
                [&](const ::xeus::xmessage& msg)
                {
                    auto const& method = msg.content()["data"]["method"];

                    nl::json states = nl::json::object();
                    xeus::buffer_sequence buffers{};
                    std::vector<nl::json> buffer_paths{};
                    serialize_all_states(states, buffers, buffer_paths);

                    nl::json metadata = {{"version", XWIDGETS_PROTOCOL_VERSION}};

                    nl::json data = nl::json::object();
                    data["method"] = "update_states";
                    data["states"] = std::move(states);
                    data["buffer_paths"] = std::move(buffer_paths);

                    registered_comm.send(std::move(metadata), std::move(data), std::move(buffers));
                }
            );

            registered_comm.on_close(
                [&](const ::xeus::xmessage&)
                {
                    // This is not trivial. The comm is destructed from within one of its method.
                    // This works because no other instruction are executed by Xeus afterwards.
                    comm_registry.erase(registered_comm.id());
                }
            );
        }

        const char* get_control_target_name()
        {
            return "jupyter.widget.control";
        }

        /**
         * Register the ``jupyter.widget.control`` Xeus target.
         *
         * This target is used by the frontend to get the state of all widget in a single message
         * (_e.g._ when restarting).
         */
        void register_control_target()
        {
            xeus::get_interpreter().comm_manager().register_comm_target(
                /** The target name */
                get_control_target_name(),
                /** Callback for comm opened by the frontend on this target */
                control_comm_opened
            );
        }

        // Making a dummy static variable to call the registration at load time.
        static const auto initialized = []()
        {
            register_control_target();
            return true;
        }();
    }
}

In [3]:
xw::slider<double> slider;

In [4]:
slider               // If the semicolon is ommitted in the last line, the return value is displayed.

A Jupyter widget with unique id: 84ceeb5dd2e8426dafd20557bba5da1c

In [5]:
slider.value = 20;      // Modifying properties of widgets triggers the update of the frontend.

20.000000

In [6]:
slider.value()          // Reading the value requires using the call operator

20.000000

In [7]:
// changine some more properties
slider.max = 40;
slider.style().handle_color = "blue";
slider.orientation = "vertical";
slider.description = "A slider";

In [8]:
#include "xcpp/xdisplay.hpp"

using xcpp::display;

In [9]:
display(slider);       // xcpp::display can be called to explicitely trigger a the display of an object.

A Jupyter widget with unique id: 84ceeb5dd2e8426dafd20557bba5da1c

## Using operator chaining to mimic keyword arguments

In [10]:
auto other_slider = xw::slider<double>::initialize()
    .min(-1.0)
    .max(1.0)
    .description("Another slider")
    .finalize();

display(other_slider);

A Jupyter widget with unique id: a08687c1effd43e08cde3fa019d28eb3

## Progress

In [11]:
#include "xwidgets/xprogress.hpp"

In [12]:
xw::progress<double> progress;

In [13]:
progress

A Jupyter widget with unique id: 136670db865c470ba658df26480a0554

In [14]:
progress.value = 60;

In [15]:
progress.style().bar_color = "red";

In [16]:
progress.description = "Completion";

In [17]:
progress.style().description_width = "30px"

"30px"

## Numerical input

In [18]:
#include "xwidgets/xnumeral.hpp"

input_line_32:1:10: fatal error: 'xwidgets/xnumeral.hpp' file not found
#include "xwidgets/xnumeral.hpp"
         ^~~~~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

In [19]:
xw::numeral<double> numeral;
numeral

input_line_33:2:6: error: no member named 'numeral' in namespace 'xw'
 xw::numeral<double> numeral;
 ~~~~^
input_line_33:2:20: error: expected '(' for function-style cast or type construction
 xw::numeral<double> numeral;
             ~~~~~~^
input_line_33:2:22: error: use of undeclared identifier 'numeral'
 xw::numeral<double> numeral;
                     ^
input_line_33:3:1: error: use of undeclared identifier 'numeral'
numeral
^


Interpreter Error: 

In [20]:
numeral.value = 4

input_line_34:2:2: error: use of undeclared identifier 'numeral'
 numeral.value = 4
 ^


Interpreter Error: 

In [ ]:
numeral.value()

## Timer

In [21]:
#include "xwidgets/xplay.hpp"

In [22]:
xw::play play;
play

A Jupyter widget with unique id: d6ecb49561bc4e32afdbfca765fce921

# Boolean widgets

## Checkbox

In [23]:
#include "xwidgets/xcheckbox.hpp"

In [24]:
xw::checkbox checkbox;

In [25]:
checkbox

A Jupyter widget with unique id: cbe4852905bf450f84bdbd434314566c

In [26]:
checkbox.value = true;

In [27]:
checkbox.indent = false;

## Toggle button

In [28]:
#include "xwidgets/xtogglebutton.hpp"

In [29]:
xw::togglebutton toggle;

In [30]:
toggle

A Jupyter widget with unique id: 424edcb1e66a4908a681d66abab48613

In [ ]:
toggle.value = true;

In [ ]:
toggle.description = "toggle";

## Valid check

In [ ]:
#include "xwidgets/xvalid.hpp"

In [ ]:
xw::valid valid;

In [ ]:
valid

In [ ]:
valid.value = true;

# String widgets

## Label widget

In [ ]:
#include "xwidgets/xlabel.hpp"

In [ ]:
xw::label label;

In [ ]:
label

In [ ]:
label.value = "Some caption";

## HTML widget

In [ ]:
#include "xwidgets/xhtml.hpp"

In [ ]:
xw::html html;

In [ ]:
html.value = R"xhtml(
    <div style="
        width: 50%;
        height: 100px;
        background: #323;
        color: white;
        text-align: center;"
        >Some HTML
    </div>
)xhtml";
html

## Text widget

In [ ]:
#include <iostream>
#include "xwidgets/xtext.hpp"

In [ ]:
xw::text text;
text.value = "Some text";
text

In [ ]:
void submit_callback()
{
    std::cout << "submitted" << std::endl;
}

In [ ]:
text.on_submit(submit_callback);

## Textarea widget

In [ ]:
#include "xwidgets/xtextarea.hpp"

In [ ]:
xw::textarea textarea;
textarea.value = R"textarea(Lorem ipsum dolor sit amet, consectetur 
adipiscing elit,  sed do eiusmod tempor incididunt ut labore et dolore
magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco
laboris  nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor
in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla 
pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa 
qui officia deserunt mollit anim id est laborum.
)textarea";
textarea

## Password widget

In [ ]:
#include "xwidgets/xpassword.hpp"

In [ ]:
xw::password password;

In [ ]:
password

# Button widget

In [ ]:
#include <iostream>
#include "xwidgets/xbutton.hpp"

In [ ]:
xw::button bt;

In [ ]:
void foo()
{
    std::cout << "Clicked!" << std::endl;
}

In [ ]:
bt.on_click(foo);
    
bt

In [ ]:
bt.description = "button";

In [ ]:
bt.button_style = "success";

In [ ]:
bt.button_style = "some invalid value";  // values are validated upon assignment

In [ ]:
std::cout << bt.button_style();

# Widgets layout

In [ ]:
bt.layout().width = "50%";
bt.layout().height = "200px";

In [ ]:
bt.style().button_color = "#888";

# Value semantics

In [ ]:
xw::button bt_copy = bt;

In [ ]:
bt_copy

In [ ]:
bt.style().button_color = "red";
bt_copy.style().button_color = "green";

In [ ]:
#include "xwidgets/xslider.hpp"

In [ ]:
xw::slider<double> slide1;
slide1.value = 4.0;

In [ ]:
xw::slider<double> slide2 = slide1;

In [ ]:
slide2

# Link widget

In [ ]:
#include "xwidgets/xslider.hpp"
#include "xwidgets/xlink.hpp"
#include "xwidgets/xbox.hpp"

In [ ]:
xw::slider<double> s1, s2;

s1.description = "Slider 1";
s2.description = "Slider 2";

In [ ]:
auto l = xw::link(s1, "value", s2, "value");

In [ ]:
s1

In [ ]:
s2

In [ ]:
xw::slider<double> source, target;

In [ ]:
auto dl = xw::directional_link(source, "value", target, "value");

In [ ]:
source

In [ ]:
target

# Box widgets

In [ ]:
#include "xwidgets/xbutton.hpp"
#include "xwidgets/xslider.hpp"
#include "xwidgets/xbox.hpp"

In [ ]:
xw::vbox b;
xw::slider<double> slid1;
slid1.description = "Slider 1";
xw::slider<double> slid2;
slid2.description = "Slider 2";

In [ ]:
b.add(xw::button());
b.add(slid1);
b.add(slid2);

In [ ]:
b

In [ ]:
b.remove(slid1)

In [ ]:
b.clear()

## Controller

In [ ]:
#include "xwidgets/xcontroller.hpp"

In [ ]:
xw::controller c

In [ ]:
c

# Selection widgets

## Toggle buttons

In [ ]:
#include "xwidgets/xtogglebuttons.hpp"

In [ ]:
xw::togglebuttons tb(std::vector<std::string>({"foo", "bar"}), "foo");

In [ ]:
tb

In [ ]:
tb.value = "bar";

In [ ]:
tb._options_labels = std::vector<std::string>({"baz", "taz"});

## Dropdown

In [ ]:
#include "xwidgets/xdropdown.hpp"

In [ ]:
xw::dropdown dd(std::vector<std::string>({"foo", "bar"}), "foo");

In [ ]:
dd

## RadioButtons

In [ ]:
#include "xwidgets/xradiobuttons.hpp"

In [ ]:
xw::radiobuttons rb(std::vector<std::string>({"foo", "bar"}), "foo");

In [ ]:
rb

## Select

In [ ]:
#include "xwidgets/xselect.hpp"

In [ ]:
xw::select sel(std::vector<std::string>({"foo", "bar"}), "foo");

In [ ]:
sel

In [ ]:
sel.rows = 3

## Selection slider

In [ ]:
#include "xwidgets/xselectionslider.hpp"

In [ ]:
xw::selectionslider sslid(std::vector<std::string>({"foo", "bar", "baz", "taz"}), "foo");

In [ ]:
sslid

## Multiple Select

In [ ]:
#include "xwidgets/xselect.hpp"

In [ ]:
xw::select_multiple mul_sel(std::vector<std::string>({"foo", "bar"}));

In [ ]:
mul_sel

In [ ]:
mul_sel.value()

In [ ]:
mul_sel.value = std::vector<std::string>();

## Selection range slider

In [ ]:
#include "xwidgets/xselectionslider.hpp"

In [ ]:
xw::selection_rangeslider range_sslid(std::vector<std::string>({"foo", "bar", "baz", "taz"}));

In [ ]:
range_sslid

In [ ]:
range_sslid.value()

# Selection Containers

## Tabs

In [ ]:
#include "xwidgets/xtab.hpp"
#include "xwidgets/xbutton.hpp"
#include "xwidgets/xslider.hpp"

In [ ]:
xw::tab tabs;

In [ ]:
xw::slider<double> tab_slid;
tabs.add(xw::button());
tabs.add(tab_slid);

In [ ]:
tabs

In [ ]:
tabs.set_title(0, "zero");
tabs.set_title(1, "one");

## Accordion

In [ ]:
#include "xwidgets/xaccordion.hpp"
#include "xwidgets/xbutton.hpp"

In [ ]:
xw::accordion accord;

In [ ]:
accord.add(xw::button());
accord.add(xw::button());
accord.add(xw::button());

In [ ]:
accord

In [ ]:
accord.set_title(0, "zero");
accord.set_title(1, "one");
accord.set_title(2, "two");

# Color picker

In [ ]:
#include "xwidgets/xcolor_picker.hpp"

In [ ]:
xw::color_picker cpicker;

In [ ]:
cpicker

In [ ]:
cpicker.value = "blue";

In [ ]:
cpicker.concise = true;

# Media

In [ ]:
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [ ]:
auto im = xw::image_from_file("marie.png").finalize();
im

In [ ]:
auto vid1 = xw::video_from_file("Big.Buck.Bunny.mp4").finalize();
vid1

In [ ]:
auto vid2 = xw::video_from_url("https://webrtc.github.io/samples/src/video/chrome.webm").finalize();
vid2

In [ ]:
auto au = xw::audio_from_file("Big.Buck.Bunny.mp3").finalize();
au

# Output

In [ ]:
#include "xwidgets/xoutput.hpp"

In [ ]:
xw::output out;
out

In [ ]:
#include <iostream>

In [ ]:
{
    // Using a scope guard to enable output capture
    auto g = out.guard();
    std::cout << "This output is captured." << std::endl;
}

In [ ]:
#include <xcpp/xdisplay.hpp>

In [ ]:
{
    // Using a scope guard to clear output widget
    auto g = out.guard();
    xcpp::clear_output();
}